In [ ]:
  if (not args.fold or args.dataset == 'bioasq'):
        args.fold = ['']
    elif args.fold == 'all':
        args.fold = ['1','2','3','4','5']
#         args.fold = ['1']
    else:
        args.fold = [args.fold]
    
    # Generate all features for robust
    # Later, divide them according to fold and data_split
    if args.dataset == 'robust':
        pass
        gen_features_param_file = workdir + dataset + '_gen_features_params'
        out_features_file = workdir + dataset + '_features'

        [all_dict_queries_file, run_filename_all, qrels_all_file, trec_query_all_file] = join_files()
        
        features_params =[
                trec_query_all_file,
                index_dir,
                out_features_file,
                run_filename_all,
                qrels_all_file,
                'krovetz', # This should not be here!, Fix GenerateExtraFeatures.cpp to read from index manifest
        ]   
        
        generate_features_params(features_params, gen_features_param_file)

        # Generate L2R features 
        feature_generator = GenerateExtraFeatures(ir_toolkit_location, gen_features_param_file)
        feature_generator.run()
        
        # Get all features in a dict
        feat_dic = features_dict(out_features_file)
    
    
    # Get features for every fold and data_split
    
    for f in args.fold:
        
        fold = f # '1'
        
        if args.dataset == 'bioasq':
            fold_dir = workdir
        else:
            fold_dir = workdir + 's' + fold + '/'
        utils.create_dir(fold_dir)
    #     # Generate qrels and qret

        if args.data_split == 'all':
            data_splits = ['train', 'dev', 'test']
        else:
            data_splits = [args.data_split]

        for data_split in data_splits:
            
            if args.dataset == 'bioasq':
                queries_file = '../../bioasq_data/bioasq.' + data_split + '.json'
#                 prefix = queries_file.split('/')[-1].strip('.json')
                prefix = dataset + '_' + data_split
                filename_prefix = workdir + prefix
                trec_query_file = filename_prefix + '_trec_query'
                run_filename = workdir + 'run_bm25_' + prefix
                run_filename_filtered = run_filename + '_filtered'
                qrels_file = filename_prefix + '_qrels'
                gen_features_param_file = filename_prefix + '_gen_features_params'
                out_features_file = filename_prefix + '_features'
                features_params =[
                    trec_query_file,
                    index_dir,
                    out_features_file,
                    run_filename_filtered,
                    qrels_file,
                    'krovetz', # This should not be here!, Fix GenerateExtraFeatures.cpp to read from index manifest
                ]    
                generate_features_params(features_params, gen_features_param_file)

                    # Generate L2R features 

                feature_generator = GenerateExtraFeatures(ir_toolkit_location, gen_features_param_file)
                feature_generator.run()
                

            elif args.dataset == 'robust':
                queries_file = '../../robust04_data/split_' + fold + '/rob04.' +  data_split + '.s' + fold + '.json'
#                 q_file = queries_file
#                 prefix = queries_file.split('/')[-1].strip('.json')
                prefix = dataset + '_' + data_split + '_s' + fold 
                filename_prefix = fold_dir + prefix

                query_list = load_queries(queries_file)
                qid_list = [q['id'] for q in query_list] 

#                 print(filename_prefix)
                out_features_split_file = filename_prefix + '_features'

                print(len(feat_dic))
                print(len(qid_list))
                
                save_features(feat_dic, qid_list, out_features_split_file)